# Base Data Building

## Purpose:
* Load and clean both Google mobility data and VDH Covid 19 data
* Merge both datasets on locality and date

## Dependencies

In [126]:
import pandas as pd

In [127]:
## Load in Google mobility data for US
gm0 = pd.read_csv("data/raw/2020_US_Region_Mobility_Report.csv")

## Check top
gm0.head()

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,US,United States,NaN,NaN,NaN,NaN,NaN,2020-02-15,6.0,2.0,15.0,3.0,2.0,-1.0
1,US,United States,NaN,NaN,NaN,NaN,NaN,2020-02-16,7.0,1.0,16.0,2.0,0.0,-1.0
2,US,United States,NaN,NaN,NaN,NaN,NaN,2020-02-17,6.0,0.0,28.0,-9.0,-24.0,5.0
3,US,United States,NaN,NaN,NaN,NaN,NaN,2020-02-18,0.0,-1.0,6.0,1.0,0.0,1.0
4,US,United States,NaN,NaN,NaN,NaN,NaN,2020-02-19,2.0,0.0,8.0,1.0,1.0,0.0


In [128]:
## Load in VDH COVID-19 Data
cv0 = pd.read_csv('data/Raw/VDH-COVID-19-PublicUseDataset-Cases_20201129.csv')

cv0.head()

,Report Date,FIPS,Locality,VDH Health District,Total Cases,Hospitalizations,Deaths
0,03/17/2020,51001,Accomack,Eastern Shore,0,0,0
1,03/17/2020,51003,Albemarle,Thomas Jefferson,0,0,0
2,03/17/2020,51005,Alleghany,Alleghany,0,0,0
3,03/17/2020,51007,Amelia,Piedmont,0,0,0
4,03/17/2020,51009,Amherst,Central Virginia,0,0,0


In [129]:
## Check frequency of state values
gm0['sub_region_1'].value_counts()

Texas                   52795
Georgia                 37191
Virginia                33236
North Carolina          26527
Kentucky                25611
Missouri                25562
Indiana                 24572
Illinois                24435
Ohio                    24382
Tennessee               24057
Iowa                    22747
Michigan                21173
Minnesota               20376
Wisconsin               18657
Mississippi             18550
Florida                 18481
Pennsylvania            18293
Alabama                 17572
New York                17346
Arkansas                17207
Oklahoma                17062
Louisiana               15876
California              15701
Kansas                  14891
South Carolina          12684
Colorado                12403
Nebraska                12095
West Virginia           11474
Washington               9893
Oregon                   9233
Idaho                    8986
New Mexico               7881
Montana                  7298
South Dako

In [130]:
## Subset to Virginia records only
gm1 = gm0.loc[gm0['sub_region_1'] == "Virginia",].copy()

## Check that dimensions match frequency count
gm1.shape

## Subset to locality records only - remove state level data
gm2 = gm1.loc[gm1['sub_region_2'].notnull(),].copy()

In [131]:
## Check on Google Mobility dates
gm2['date'].dtypes

## Standardize date format
gm2['date'] = pd.to_datetime(gm1['date'], format='%Y-%m-%d')

## Print min/max
print(
min(gm2['date']),
max(gm2['date'])
)

2020-02-15 00:00:00 2020-11-24 00:00:00


In [132]:
## Check on COVID-19 dates
## Print min/max
print(
min(cv0['Report Date']),
max(cv0['Report Date'])
)

03/17/2020 11/29/2020


In [133]:
## Standardize date format
cv0['Report Date'] = pd.to_datetime(cv0['Report Date'], format='%m/%d/%Y')

## Print min/max
print(
min(cv0['Report Date']),
max(cv0['Report Date'])
)

2020-03-17 00:00:00 2020-11-29 00:00:00


In [134]:
## Subset data to start on March 17 (start of VA COVID-19 Data) to November 24 (end of mobility data)
gm3 = gm2.loc[pd.Timestamp(2020,3,17) <= gm2['date'],].copy()

## Print min/max after subset
print(
min(gm3['date']),
max(gm3['date'])
)

2020-03-17 00:00:00 2020-11-24 00:00:00


In [135]:
## Subset data to start on March 17 (start of VA COVID-19 Data) to November 24 (end of mobility data)
cv1 = cv0.loc[cv0['Report Date'] <= pd.Timestamp(2020,11,24),].copy()

## Print min/max after subset
print(
min(cv1['Report Date']),
max(cv1['Report Date'])
)

2020-03-17 00:00:00 2020-11-24 00:00:00


In [136]:
## Check GM locality values
gm2['sub_region_2'].value_counts()

# [NOTE - Locality values are not consistent]

Alexandria             284
Spotsylvania County    284
Hanover County         284
Harrisonburg           284
Albemarle County       284
                      ... 
Rappahannock County    149
Craig County           129
Bath County            111
Norton                  82
Lexington               67
Name: sub_region_2, Length: 132, dtype: int64

In [137]:
## Check CV locality values
cv1['Locality'].value_counts()

Rockbridge       253
Prince Edward    253
Nelson           253
Spotsylvania     253
Pulaski          253
                ... 
Falls Church     253
Staunton         253
Scott            253
Roanoke City     253
Craig            253
Name: Locality, Length: 133, dtype: int64

In [138]:
## Check all GM locality values
gm3['sub_region_2'].unique()

array(['Accomack County', 'Albemarle County', 'Alexandria',
       'Alleghany County', 'Amelia County', 'Amherst County',
       'Appomattox County', 'Arlington County', 'Augusta County',
       'Bath County', 'Bedford County', 'Bland County',
       'Botetourt County', 'Bristol', 'Brunswick County',
       'Buchanan County', 'Buckingham County', 'Buena Vista',
       'Campbell County', 'Caroline County', 'Carroll County',
       'Charles City County', 'Charlotte County', 'Charlottesville',
       'Chesapeake', 'Chesterfield County', 'Clarke County',
       'Colonial Heights', 'Covington', 'Craig County', 'Culpeper County',
       'Cumberland County', 'Danville', 'Dickenson County',
       'Dinwiddie County', 'Emporia', 'Essex County', 'Fairfax',
       'Fairfax County', 'Falls Church', 'Fauquier County',
       'Floyd County', 'Fluvanna County', 'Franklin', 'Franklin County',
       'Frederick County', 'Fredericksburg', 'Galax', 'Giles County',
       'Gloucester County', 'Goochland C

In [139]:
## Check all CV locality values
cv1['Locality'].unique()

array(['Accomack', 'Albemarle', 'Alleghany', 'Amelia', 'Amherst',
       'Appomattox', 'Arlington', 'Augusta', 'Bath', 'Bedford', 'Bland',
       'Botetourt', 'Brunswick', 'Buchanan', 'Buckingham', 'Campbell',
       'Caroline', 'Carroll', 'Charles City', 'Charlotte', 'Chesterfield',
       'Clarke', 'Craig', 'Culpeper', 'Cumberland', 'Dickenson',
       'Dinwiddie', 'Essex', 'Fairfax', 'Fauquier', 'Floyd', 'Fluvanna',
       'Franklin County', 'Frederick', 'Giles', 'Gloucester', 'Goochland',
       'Grayson', 'Greene', 'Greensville', 'Halifax', 'Hanover',
       'Henrico', 'Henry', 'Highland', 'Isle of Wight', 'James City',
       'King and Queen', 'King George', 'King William', 'Lancaster',
       'Lee', 'Loudoun', 'Louisa', 'Lunenburg', 'Madison', 'Mathews',
       'Mecklenburg', 'Middlesex', 'Montgomery', 'Nelson', 'New Kent',
       'Northampton', 'Northumberland', 'Nottoway', 'Orange', 'Page',
       'Patrick', 'Pittsylvania', 'Powhatan', 'Prince Edward',
       'Prince George', 

In [140]:
## Export for review in CSV files
gm3.to_csv("data/build/build1_gm_subset.csv", index=False)
cv1.to_csv("data/build/build1_cv_subset.csv", index=False)

In [141]:
## Standardize locality values across data sets

## Add "City" where necessary
gm3.loc[gm3['sub_region_2'] == "Buena Vista", 'sub_region_2'] = "Buena Vista City"
gm3.loc[gm3['sub_region_2'] == "Fairfax", 'sub_region_2']     = "Fairfax City"
gm3.loc[gm3['sub_region_2'] == "Franklin", 'sub_region_2']    = "Franklin City"
gm3.loc[gm3['sub_region_2'] == "Manassas", 'sub_region_2']    = "Manassas City"
gm3.loc[gm3['sub_region_2'] == "Richmond", 'sub_region_2']    = "Richmond City"
gm3.loc[gm3['sub_region_2'] == "Roanoke", 'sub_region_2']     = "Roanoke City"

## Check on change
gm3.loc[gm3['sub_region_2'] == "Buena Vista City",]

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
641033,US,United States,Virginia,Buena Vista City,NaN,NaN,51530.0,2020-03-17,NaN,NaN,NaN,NaN,-12.0,NaN
641034,US,United States,Virginia,Buena Vista City,NaN,NaN,51530.0,2020-03-18,NaN,NaN,NaN,NaN,-22.0,NaN
641035,US,United States,Virginia,Buena Vista City,NaN,NaN,51530.0,2020-03-19,NaN,NaN,NaN,NaN,-23.0,NaN
641036,US,United States,Virginia,Buena Vista City,NaN,NaN,51530.0,2020-03-20,-29.0,NaN,NaN,NaN,-30.0,NaN
641037,US,United States,Virginia,Buena Vista City,NaN,NaN,51530.0,2020-03-23,NaN,NaN,NaN,NaN,-28.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641190,US,United States,Virginia,Buena Vista City,NaN,NaN,51530.0,2020-11-18,NaN,NaN,NaN,NaN,-27.0,NaN
641191,US,United States,Virginia,Buena Vista City,NaN,NaN,51530.0,2020-11-19,NaN,NaN,NaN,NaN,-22.0,NaN
641192,US,United States,Virginia,Buena Vista City,NaN,NaN,51530.0,2020-11-20,NaN,NaN,NaN,NaN,-27.0,NaN
641193,US,United States,Virginia,Buena Vista City,NaN,NaN,51530.0,2020-11-23,NaN,NaN,NaN,NaN,-29.0,NaN


In [143]:
## Remove " County"
gm3['sub_region_2'] = gm3['sub_region_2'].str.replace(' County','')
cv1['Locality'] = cv1['Locality'].str.replace(' County','')

In [144]:
## Pre-Merge checks
print(
gm3.shape,
cv1.shape
)

(29019, 14) (33649, 7)


In [145]:
base = cv1.merge(gm3, how= "outer", left_on = ["Report Date", "Locality"],
                                    right_on= ["date","sub_region_2"])

base.head()

,Report Date,FIPS,Locality,VDH Health District,Total Cases,Hospitalizations,Deaths,country_region_code,country_region,sub_region_1,...,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,2020-03-17,51001,Accomack,Eastern Shore,0,0,0,US,United States,Virginia,...,NaN,NaN,51001.0,2020-03-17,-8.0,11.0,NaN,NaN,-15.0,5.0
1,2020-03-17,51003,Albemarle,Thomas Jefferson,0,0,0,US,United States,Virginia,...,NaN,NaN,51003.0,2020-03-17,-22.0,9.0,NaN,-16.0,-38.0,14.0
2,2020-03-17,51005,Alleghany,Alleghany,0,0,0,US,United States,Virginia,...,NaN,NaN,51005.0,2020-03-17,-11.0,NaN,NaN,8.0,-12.0,NaN
3,2020-03-17,51007,Amelia,Piedmont,0,0,0,US,United States,Virginia,...,NaN,NaN,51007.0,2020-03-17,-21.0,6.0,NaN,NaN,-18.0,NaN
4,2020-03-17,51009,Amherst,Central Virginia,0,0,0,US,United States,Virginia,...,NaN,NaN,51009.0,2020-03-17,3.0,22.0,NaN,NaN,-13.0,6.0


In [147]:
## Post-Merge checks
base.shape

(33649, 21)

In [146]:
## Export for review
base.to_csv("data/build/build1_base.csv", index=False)